<a href="https://colab.research.google.com/github/abhishek96negi/Machine-Learning/blob/main/Algorithm/Decision%20Tree/Example/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Predicting Survival in the Titanic Data Set
We will be using a decision tree to make predictions about the Titanic data
set from Kaggle. This data set provides information on the Titanic
passengers and can be used to predict whether a passenger survived or
not.

Loading Data and modules


In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler

In [ ]:
url='https://raw.githubusercontent.com/abhishek96negi/DataSet/master/Titanic.csv'

titanic = pd.read_csv(url)
titanic

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
titanic.drop('name',axis=1,inplace=True)

In [ ]:
titanic.isnull().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
ticket        0
fare          0
cabin       687
embarked      2
dtype: int64

In [ ]:
titanic.drop('cabin',axis=1,inplace=True)

In [ ]:
titanic.isnull().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
ticket        0
fare          0
embarked      2
dtype: int64

In [ ]:
titanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
titanic['age'].dropna().sample(titanic['age'].isnull().sum(),random_state=0)

423    28.00
177    50.00
305     0.92
292    36.00
889    26.00
       ...  
539    22.00
267    25.00
352    15.00
99     34.00
689    15.00
Name: age, Length: 177, dtype: float64

In [ ]:
def impute_nan(df,variable,median):
    df[variable+"_median"]=df[variable].fillna(median)
    df[variable+"_random"]=df[variable]
    ##It will have the random sample to fill the na
    random_sample=df[variable].dropna().sample(df[variable].isnull().sum(),random_state=0)
    ##pandas need to have same index in order to merge the dataset
    random_sample.index=df[df[variable].isnull()].index
    df.loc[df[variable].isnull(),variable+'_random']=random_sample

In [ ]:
median=titanic.age.median()
median

28.0

In [ ]:
impute_nan(titanic,'age',median)

In [ ]:
titanic.isnull().sum()

survived        0
pclass          0
sex             0
age           177
sibsp           0
parch           0
ticket          0
fare            0
embarked        2
age_median      0
age_random      0
dtype: int64

In [ ]:
titanic.drop('age',axis=1,inplace=True)

In [ ]:
titanic.isnull().sum()

survived      0
pclass        0
sex           0
sibsp         0
parch         0
ticket        0
fare          0
embarked      2
age_median    0
age_random    0
dtype: int64

In [ ]:
def impute_nan(df,variable):
    most_frequent_category=df[variable].mode()[0]
    df[variable].fillna(most_frequent_category,inplace=True)

In [ ]:
impute_nan(titanic,'embarked')

In [ ]:
titanic.isnull().sum()

survived      0
pclass        0
sex           0
sibsp         0
parch         0
ticket        0
fare          0
embarked      0
age_median    0
age_random    0
dtype: int64

In [ ]:
titanic['embarked'].unique()

array(['S', 'C', 'Q'], dtype=object)

In [ ]:
data=pd.get_dummies(titanic['embarked'],drop_first=True)
data

,Q,S
0,0,1
1,0,0
2,0,1
3,0,1
4,0,1
...,...,...
886,0,1
887,0,1
888,0,1
889,0,0


In [ ]:
titanic

,survived,pclass,sex,sibsp,parch,ticket,fare,embarked,age_median,age_random
0,0,3,male,1,0,A/5 21171,7.2500,S,22.0,22.0
1,1,1,female,1,0,PC 17599,71.2833,C,38.0,38.0
2,1,3,female,0,0,STON/O2. 3101282,7.9250,S,26.0,26.0
3,1,1,female,1,0,113803,53.1000,S,35.0,35.0
4,0,3,male,0,0,373450,8.0500,S,35.0,35.0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,0,0,211536,13.0000,S,27.0,27.0
887,1,1,female,0,0,112053,30.0000,S,19.0,19.0
888,0,3,female,1,2,W./C. 6607,23.4500,S,28.0,15.0
889,1,1,male,0,0,111369,30.0000,C,26.0,26.0


In [ ]:
new_data=pd.merge(left=titanic, right=data, how='outer',left_index=True,right_index=True)
new_data

,survived,pclass,sex,sibsp,parch,ticket,fare,embarked,age_median,age_random,Q,S
0,0,3,male,1,0,A/5 21171,7.2500,S,22.0,22.0,0,1
1,1,1,female,1,0,PC 17599,71.2833,C,38.0,38.0,0,0
2,1,3,female,0,0,STON/O2. 3101282,7.9250,S,26.0,26.0,0,1
3,1,1,female,1,0,113803,53.1000,S,35.0,35.0,0,1
4,0,3,male,0,0,373450,8.0500,S,35.0,35.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,0,0,211536,13.0000,S,27.0,27.0,0,1
887,1,1,female,0,0,112053,30.0000,S,19.0,19.0,0,1
888,0,3,female,1,2,W./C. 6607,23.4500,S,28.0,15.0,0,1
889,1,1,male,0,0,111369,30.0000,C,26.0,26.0,0,0


In [ ]:
new_data.drop('embarked',axis=1,inplace=True)

In [ ]:
new_data

,survived,pclass,sex,sibsp,parch,ticket,fare,age_median,age_random,Q,S
0,0,3,male,1,0,A/5 21171,7.2500,22.0,22.0,0,1
1,1,1,female,1,0,PC 17599,71.2833,38.0,38.0,0,0
2,1,3,female,0,0,STON/O2. 3101282,7.9250,26.0,26.0,0,1
3,1,1,female,1,0,113803,53.1000,35.0,35.0,0,1
4,0,3,male,0,0,373450,8.0500,35.0,35.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,0,0,211536,13.0000,27.0,27.0,0,1
887,1,1,female,0,0,112053,30.0000,19.0,19.0,0,1
888,0,3,female,1,2,W./C. 6607,23.4500,28.0,15.0,0,1
889,1,1,male,0,0,111369,30.0000,26.0,26.0,0,0


In [ ]:
titanic['sex'].unique()

array(['male', 'female'], dtype=object)

In [ ]:
data=pd.get_dummies(titanic['sex'],drop_first=True)
data

,male
0,1
1,0
2,0
3,0
4,1
...,...
886,1
887,0
888,0
889,1


In [ ]:
new_data=pd.merge(left=new_data, right=data, how='outer',left_index=True,right_index=True)
new_data

,survived,pclass,sex,sibsp,parch,ticket,fare,age_median,age_random,Q,S,male
0,0,3,male,1,0,A/5 21171,7.2500,22.0,22.0,0,1,1
1,1,1,female,1,0,PC 17599,71.2833,38.0,38.0,0,0,0
2,1,3,female,0,0,STON/O2. 3101282,7.9250,26.0,26.0,0,1,0
3,1,1,female,1,0,113803,53.1000,35.0,35.0,0,1,0
4,0,3,male,0,0,373450,8.0500,35.0,35.0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,0,0,211536,13.0000,27.0,27.0,0,1,1
887,1,1,female,0,0,112053,30.0000,19.0,19.0,0,1,0
888,0,3,female,1,2,W./C. 6607,23.4500,28.0,15.0,0,1,0
889,1,1,male,0,0,111369,30.0000,26.0,26.0,0,0,1


In [ ]:
new_data.drop('sex',axis=1,inplace=True)

In [ ]:
new_data.drop('ticket',axis=1,inplace=True)

In [ ]:
new_data

,survived,pclass,sibsp,parch,fare,age_median,age_random,Q,S,male
0,0,3,1,0,7.2500,22.0,22.0,0,1,1
1,1,1,1,0,71.2833,38.0,38.0,0,0,0
2,1,3,0,0,7.9250,26.0,26.0,0,1,0
3,1,1,1,0,53.1000,35.0,35.0,0,1,0
4,0,3,0,0,8.0500,35.0,35.0,0,1,1
...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,0,13.0000,27.0,27.0,0,1,1
887,1,1,0,0,30.0000,19.0,19.0,0,1,0
888,0,3,1,2,23.4500,28.0,15.0,0,1,0
889,1,1,0,0,30.0000,26.0,26.0,0,0,1


In [ ]:
X=new_data.drop('survived',axis=1)
X

,pclass,sibsp,parch,fare,age_median,age_random,Q,S,male
0,3,1,0,7.2500,22.0,22.0,0,1,1
1,1,1,0,71.2833,38.0,38.0,0,0,0
2,3,0,0,7.9250,26.0,26.0,0,1,0
3,1,1,0,53.1000,35.0,35.0,0,1,0
4,3,0,0,8.0500,35.0,35.0,0,1,1
...,...,...,...,...,...,...,...,...,...
886,2,0,0,13.0000,27.0,27.0,0,1,1
887,1,0,0,30.0000,19.0,19.0,0,1,0
888,3,1,2,23.4500,28.0,15.0,0,1,0
889,1,0,0,30.0000,26.0,26.0,0,0,1


In [ ]:
y=new_data['survived']
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: survived, Length: 891, dtype: int64

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.30, random_state= 355)

In [ ]:
#let's first visualize the tree on the data without doing any pre processing
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
feature_name=list(X.columns)
feature_name

['pclass',
 'sibsp',
 'parch',
 'fare',
 'age_median',
 'age_random',
 'Q',
 'S',
 'male']

In [ ]:
class_name = list(y_train.unique())
class_name

[1, 0]

In [ ]:
clf.score(x_train,y_train)

0.9919743178170144

In [ ]:
py_pred = clf.predict(x_test)
py_pred

array([1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1])

In [ ]:
# accuracy of our classification tree
clf.score(x_test,y_test)

0.746268656716418

In [ ]:
scalar = StandardScaler()
scalar

StandardScaler(copy=True, with_mean=True, with_std=True)

In [ ]:
x_transform = scalar.fit_transform(X)
x_transform

array([[ 0.82737724,  0.43279337, -0.47367361, ..., -0.30756234,
         0.61583843,  0.73769513],
       [-1.56610693,  0.43279337, -0.47367361, ..., -0.30756234,
        -1.62380254, -1.35557354],
       [ 0.82737724, -0.4745452 , -0.47367361, ..., -0.30756234,
         0.61583843, -1.35557354],
       ...,
       [ 0.82737724,  0.43279337,  2.00893337, ..., -0.30756234,
         0.61583843, -1.35557354],
       [-1.56610693, -0.4745452 , -0.47367361, ..., -0.30756234,
        -1.62380254,  0.73769513],
       [ 0.82737724, -0.4745452 , -0.47367361, ...,  3.25137334,
        -1.62380254,  0.73769513]])

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x_transform,y,test_size = 0.30, random_state= 355)

In [ ]:
# we are tuning three hyperparameters right now, we are passing the different values for both parameters
grid_param = {
    'criterion': ['gini', 'entropy'],
    'max_depth' : range(2,100,1),
    'min_samples_leaf' : range(1,10,1),
    'min_samples_split': range(2,10,1),
    'splitter' : ['best', 'random']
    
}
grid_param

{'criterion': ['gini', 'entropy'],
 'max_depth': range(2, 100),
 'min_samples_leaf': range(1, 10),
 'min_samples_split': range(2, 10),
 'splitter': ['best', 'random']}

In [ ]:
grid_search = GridSearchCV(estimator=clf,
                     param_grid=grid_param,
                     cv=5,
                    n_jobs =-1)
grid_search

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                     

In [ ]:
grid_search.fit(x_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                     

In [ ]:
best_parameters = grid_search.best_params_
print(best_parameters)

{'criterion': 'entropy', 'max_depth': 33, 'min_samples_leaf': 5, 'min_samples_split': 6, 'splitter': 'random'}


In [ ]:
grid_search.best_score_

0.8266580645161291

In [ ]:
clf = DecisionTreeClassifier(criterion = 'entropy', max_depth =33, min_samples_leaf= 5, min_samples_split= 6, splitter ='random')
clf.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=33, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=6,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='random')

In [ ]:
clf.score(x_test,y_test)

0.7873134328358209

In [ ]:
clf.predict([[3,	1,	0,	7.2500,	22.0,	22.0,	0	,1	,1]])

array([0])